In [1]:
import numpy as np
import torch as th
import syft as sy

In [2]:
sy.create_sandbox(globals())

Setting up Sandbox...
	- Hooking PyTorch
	- Creating Virtual Workers:
		- bob
		- theo
		- jason
		- alice
		- andy
		- jon
	Storing hook and workers as global variables...
	Loading datasets from SciKit Learn...
		- Boston Housing Dataset
		- Diabetes Dataset
		- Breast Cancer Dataset
	- Digits Dataset
		- Iris Dataset
		- Wine Dataset
		- Linnerud Dataset
	Distributing Datasets Amongst Workers...
Done!


# Ordinary least squared regression and LDLt decomposition

**Example data**: the correct $\beta$ is $[1, 2, -1]$

In [3]:
X = th.tensor(10 * np.random.randn(30, 3))
y = (X[:, 0] + 2 * X[:, 1] - X[:, 2]).reshape(-1, 1)

Split the data into chunks and send a chunk to each worker, storing pointers to chunks in two `MultiPointerTensor`s.

In [4]:
workers = [alice, bob, theo]
crypto_provider = jon
chunk_size = int(X.shape[0] / len(workers))

def _get_chunk_pointers(data, chunk_size, workers):
    return [
        data[(i * chunk_size):((i+1)*chunk_size), :].send(worker)
        for i, worker in enumerate(workers)
    ] 

X_ptrs = sy.MultiPointerTensor(
    children=_get_chunk_pointers(X, chunk_size, workers))
y_ptrs = sy.MultiPointerTensor(
    children=_get_chunk_pointers(y, chunk_size, workers))

This is the "big data" step, and it's performed locally on each worker in plain text. The result is two `MultiPointerTensor`s with pointers to each workers' summand of $X^tX$ (or $X^ty$).

In [5]:
Xt_ptrs = X_ptrs.transpose(0, 1)

XtX_summand_ptrs = Xt_ptrs.mm(X_ptrs)
Xty_summand_ptrs = Xt_ptrs.mm(y_ptrs)

We add those summands up in two steps:
- share each summand among all other workers
- move the resulting pointers to one place (here just the local worker) and add 'em up.

In [6]:
def _generate_shared_summand_pointers(
        summand_ptrs, 
        workers, 
        crypto_provider):

    for worker_id, summand_pointer in summand_ptrs.child.items():
        shared_summand_pointer = summand_pointer.fix_precision().share(
            *workers, crypto_provider=crypto_provider)
        yield shared_summand_pointer.get()

In [7]:
XtX_shared = sum(
    _generate_shared_summand_pointers(
        XtX_summand_ptrs, workers, crypto_provider))

Xty_shared = sum(_generate_shared_summand_pointers(
    Xty_summand_ptrs, workers, crypto_provider))

The coefficient $\beta$ is the solution to
$$X^t X \beta = X^t y$$

We solve for $\beta$ by 
1. Decomposing $X^t X = LDL^t$, where L is a lower triangular matrix and $D$ is a diagonal matrix (for more details, see https://en.wikipedia.org/wiki/Cholesky_decomposition#LDL_decomposition). 
2. Solving $L \alpha = X^ty$, for $\alpha$ which is straightforward since $L$ is lower-triangular.
3. Solving $D L^t \beta = \alpha$, for $\beta$, which is also straightforward since $D L^t$ is upper-triangular.

Critically, all steps are just compositions of linear operations that are supported by `AdditiveSharingTensor`. In particular, unlike the classic Cholesky decomposition, the $LDL^t$ decomposition in step 1 does not involve taking square roots, which would be challenging in the secure setting.

We implement these steps below. 

**TODO**: At the moment, `AdditiveSharingTensor` doesn't appear to support the types of indexing operations used. Seems like it should be possible to modify its `__getitem__` method to support these, but having trouble figuring out how this interacts with the hooking that's going on. Instead, will just perform the computation on the local worker.

In [8]:
def _eye(n):
    """th.eye doesn't seem to work after hooking torch, so just adding
    a workaround for now.
    """
    return th.FloatTensor(np.eye(n))


def ldlt_decomposition(x):
    """Decompose the square, symmetric, full-rank matrix X as X = LDL^t, where 
        - L is upper triangular
        - D is diagonal.
    X must be a square, symmetric matrix of full rank.
    """
    n, _ = x.shape
    l, diag = _eye(n), th.zeros(n).float()

    for j in range(n):
        diag[j] = x[j, j] - (th.sum((l[j, :j] ** 2) * diag[:j]))
        for i in range(j + 1, n):
            # instability is a concern for small d.
            l[i, j] = (x[i, j] - th.sum(diag[:j] * l[i, :j] * l[j, :j])) / diag[j]

    return l, th.diag(diag), l.transpose(0, 1)


def solve_upper_triangular(u, y):
    """Solve Ux = y for U a square, upper triangular matrix"""
    n = u.shape[0]
    x = th.zeros(n)
    for i in range(n - 1, -1, -1):
        x[i] = (y[i] - th.sum(u[i, i+1:] * x[i+1:])) / u[i, i]

    return x.reshape(-1, 1)


def solve_lower_triangular(l, y):
    """Solve Lx = y for L a square, lower triangular matrix of full rank."""
    n = l.shape[0]
    x = th.zeros(n)
    for i in range(0, n):
        x[i] = (y[i] - th.sum(l[i, :i] * x[:i])) / l[i, i]

    return x.reshape(-1, 1)


def solve_symmetric(a, y):
    """Solve the linear system Ax = y where A is a symmetric matrix of full rank."""
    l, d, lt = ldlt_decomposition(a)
    x_ = solve_lower_triangular(l.mm(d), y)
    return solve_upper_triangular(lt, x_)


In [9]:
beta = solve_symmetric(XtX_shared.get().float_precision(), Xty_shared.get().float_precision())

In [10]:
beta

tensor([[ 1.0000],
        [ 2.0000],
        [-1.0000]])

# QR-decomposition

In [11]:
"""
Full QR decomposition via Householder transforms, 
following the implementationof Numerical Linear Algebra 
(Trefethen and Bau).
"""

def _apply_householder_transform(a, v):
    return a - 2 * v.mm(v.transpose(0, 1).mm(a))


def _build_householder_matrix(v):
    n = v.shape[0]
    u = v / v.norm()
    return _eye(n) - 2 * u.mm(u.transpose(0, 1))


def _householder_qr_step(a):

    x = a[:, 0].reshape(-1, 1)
    alpha = x.norm()
    u = x.copy()

    # note: can get better stability by multiplying by sign(u[0, 0])
    # (where sign(0)n = 1); is this supported in the secure context?
    u[0, 0] += u.norm()
    
    # is there a simple way of getting around computing the norm twice?
    u /= u.norm()
    a = _apply_householder_transform(a, u)

    return a, u


def _recover_q(householder_vectors):
    """
    Build the matrix Q from the Householder transforms.
    """
    n = len(householder_vectors)

    def _apply_transforms(x):
        """Trefethen and Bau, Algorithm 10.3"""
        for k in range(n-1, -1, -1):
            x[k:, :] = _apply_householder_transform(
                x[k:, :], 
                householder_vectors[k])
        return x

    m = householder_vectors[0].shape[0]
    n = len(householder_vectors)
    q = th.zeros(m, m)
    
    # Determine q by evaluating it on a basis
    for i in range(m):
        e = th.zeros(m, 1)
        e[i] = 1.
        q[:, [i]] = _apply_transforms(e)
    
    return q


def qr(a, return_q=True):
    """
    :param a: shape (m, n), m >= n
    :return: - orthogonal q of shape (m, m), 
             - upper-triangular of shape (m, n)
    """
    m, n = a.shape
    assert m >= n, \
        f"Passed a of shape {a.shape}, must have a.shape[0] >= a.shape[1]"

    r = a.copy()
    householder_unit_normal_vectors = []

    for k in range(n):
        r[k:, k:], u = _householder_qr_step(r[k:, k:])
        householder_unit_normal_vectors.append(u)
    if return_q:
        q = _recover_q(householder_unit_normal_vectors)
    else:
        q = None
    return q, r


In [12]:
"""
Basic tests for QR decomposition
"""

def _assert_small(x, failure_msg, threshold=1E-5):
    norm = x.norm()
    assert norm < threshold, failure_msg


def _test_case(a): 
    
    # decomposition holds
    q, r = qr(a)
    _assert_small(q.mm(r) - a, "QR = A failed")

    # q is orthogonal
    m, _ = a.shape
    _assert_small(
        q.mm(q.transpose(0, 1)) - _eye(m),
        "QQ^t = I failed"
    )

    _assert_small(
        q.transpose(0, 1).mm(q) - _eye(m),
        "QQ^t = I failed"
    )
    
    # r is upper triangular
    lower_triangular_entries = th.tensor([
        r[i, j].item() for i in range(r.shape[0]) 
             for j in range(i)])

    _assert_small(
        lower_triangular_entries,
        "R is not upper triangular"
    )
    print(f"PASSED for \n{a}\n")


_test_case(
    th.tensor([[1, 0, 1],
               [1, 1, 0],
               [0, 1, 1]]).float()
)

_test_case(
    th.tensor([[1, 0, 1],
               [1, 1, 0],
               [0, 1, 1],
               [1, 1, 1],]).float()
)

PASSED for 
tensor([[1., 0., 1.],
        [1., 1., 0.],
        [0., 1., 1.]])

PASSED for 
tensor([[1., 0., 1.],
        [1., 1., 0.],
        [0., 1., 1.],
        [1., 1., 1.]])



# DASH implementation

In [15]:
n1 = 1000
n2 = 2000
n3 = 1500
m = 10000
k = 3

d = n1 + n2 + n3 - k - 1


# Alice
y1 = th.randn(n1, 1)
X1 = th.randn(n1, m)
C1 = th.randn(n1, k)
_, R1 = qr(C1)


# Bob
y2 = th.randn(n2, 1)
X2 = th.randn(n2, m)
C2 = th.randn(n2, k)
_, R2 = qr(C2)


# Carla
y3 = th.randn(n3, 1)
X3 = th.randn(n3, m)
C3 = th.randn(n3, k)
_, R3 = qr(C3)





In [14]:
R1

tensor([[-3.1591e+01,  1.6858e+00,  6.5854e-01],
        [ 2.9802e-08, -3.2810e+01,  1.1307e+00],
        [ 9.3132e-10, -3.5763e-07, -3.2311e+01],
        ...,
        [-7.4506e-09, -4.7684e-07,  3.7253e-09],
        [-1.1921e-07,  7.1526e-07, -5.9605e-08],
        [-5.9605e-08, -1.7881e-07,  1.1921e-07]])